In [0]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!wget https://raw.githubusercontent.com/google-research/bert/master/modeling.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/optimization.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/run_classifier.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py 

--2019-09-03 17:54:19--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.214.128, 2607:f8b0:4001:c05::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.214.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   233MB/s    in 1.7s    

2019-09-03 17:54:25 (233 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

--2019-09-03 17:54:26--  https://raw.githubusercontent.com/google-research/bert/master/modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37922 (37K) [tex

In [0]:
import modeling
import optimization
import run_classifier
import tokenization

W0903 17:54:35.129578 139916593944448 deprecation_wrapper.py:119] From /content/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
# 

In [0]:
!pip install tensorflow
!pip install zipfile36
import zipfile
folder = 'model_folder'
with zipfile.ZipFile("uncased_L-12_H-768_A-12.zip","r") as zip_ref:
    zip_ref.extractall(folder)
    


In [0]:
import tensorflow as tf

In [0]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = 'OUTPUT_MODEL_80'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = True #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = True #@param {type:"boolean"}
BUCKET = 'sentiment_data_dawn' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


W0903 17:59:38.655646 139916593944448 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



***** Model output directory: gs://sentiment_data_dawn/OUTPUT_MODEL_80 *****


In [0]:
BERT_MODEL = 'uncased_L-12_H-768_A-12'
BERT_PRETRAINED_DIR = f'{folder}/uncased_L-12_H-768_A-12'
# OUTPUT_DIR = f'{folder}/outputs'
print(f'>> Model output directory: {OUTPUT_DIR}')
print(f'>>  BERT pretrained directory: {BERT_PRETRAINED_DIR}')

>> Model output directory: gs://sentiment_data_dawn/OUTPUT_MODEL_80
>>  BERT pretrained directory: model_folder/uncased_L-12_H-768_A-12


In [0]:
import pandas as pd
import numpy as np
import io
from google.colab import files
uploaded = files.upload()

Saving train160k.csv to train160k.csv


In [0]:
# df = pd.read_csv('Tweets.csv', low_memory=False)
# df.shape
# emo = {'negative': 0, 'neutral': 1, 'positive': 2}
# df.airline_sentiment = [emo[item] for item in df.airline_sentiment]
# df.airline_sentiment

df = pd.read_csv('train160k.csv', encoding = 'latin', low_memory=False, names=['labels','id','date','q','usr','text'])
print(type(df.labels))
df.dropna(subset=["text"], inplace=True)

<class 'pandas.core.series.Series'>


In [0]:
# df2 = pd.DataFrame()
# df2["Text"] = df["text"]
# # df2["Label"] = LabelEncoder().fit_transform(y)
# df2["Label"] = df["airline_sentiment"]
# df2 = df2[:14640]
# df2.head()
df2 = pd.DataFrame()
df2["Text"] = df["text"]
# df2["Label"] = LabelEncoder().fit_transform(y)
df2["Label"] = df["labels"]
# df2 = df2[:14640]
# df2.head()


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df2["Text"].values, df2["Label"].values, test_size=0.2, random_state=42)
X_train = X_train[1:2001]
y_train = y_train[1:2001]
X_test = X_test[1:1001]
y_test = y_test[1:1001]
print(X_train.shape,np.sum(y_train))
print(X_test.shape,np.sum(y_test))

(2000,) 996
(1000,) 505


In [0]:
X_test

In [0]:
import os
import tensorflow as tf

def create_examples(lines, set_type, labels=None):
#Generate data for the BERT model
    guid = f'{set_type}'
    examples = []
    if guid == 'train':
        for line, label in zip(lines, labels):
            text_a = line
            label = str(label)
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    else:
        for line in lines:
            text_a = line
            label = '0'
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    return examples

# Model Hyper Parameters
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
LEARNING_RATE = 1e-5
NUM_TRAIN_EPOCHS = 3.0
WARMUP_PROPORTION = 0.1
MAX_SEQ_LENGTH = 80
# Model configs
SAVE_CHECKPOINTS_STEPS = 100000 #if you wish to finetune a model on a larger dataset, use larger interval
# each checpoint weights about 1,5gb
ITERATIONS_PER_LOOP = 100000
NUM_TPU_CORES = 8
VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
DO_LOWER_CASE = BERT_MODEL.startswith('uncased')

label_list = [str(num) for num in range(2)]
tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=DO_LOWER_CASE)
train_examples = create_examples(X_train, 'train', labels=y_train)

tpu_cluster_resolver = None #Since training will happen on GPU, we won't need a cluster resolver
#TPUEstimator also supports training on CPU and GPU. You don't need to define a separate tf.estimator.Estimator.
run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=OUTPUT_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

num_train_steps = int(
    len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

model_fn = run_classifier.model_fn_builder(
    bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
    num_labels=len(label_list),
    init_checkpoint=INIT_CHECKPOINT,
    learning_rate=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available  
    use_one_hot_embeddings=True)

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available 
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE)

W0903 19:38:41.686290 139916593944448 estimator.py:1984] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f4083d88950>) includes params argument, but params are not passed to Estimator.
W0903 19:38:41.688184 139916593944448 tpu_context.py:211] eval_on_tpu ignored because use_tpu is False.


In [0]:
import datetime
print('Please wait...')
train_features = run_classifier.convert_examples_to_features(
    train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
print('>> Started training at {} '.format(datetime.datetime.now()))
print('  Num examples = {}'.format(len(train_examples)))
print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
tf.logging.info("  Num steps = %d", num_train_steps)
train_input_fn = run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print('>> Finished training at {}'.format(datetime.datetime.now()))

W0903 19:38:45.255223 139916593944448 deprecation_wrapper.py:119] From /content/run_classifier.py:774: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.



Please wait...
>> Started training at 2019-09-03 19:38:45.768229 
  Num examples = 2000
  Batch size = 32


W0903 19:38:47.097332 139916593944448 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0903 19:38:47.691385 139916593944448 deprecation_wrapper.py:119] From /content/modeling.py:171: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0903 19:38:47.696066 139916593944448 deprecation_wrapper.py:119] From /content/modeling.py:409: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0903 19:38:47.730457 139916593944448 deprecation_wrapper.py:119] From /content/modeling.py:490: The name tf.assert_less_equal is deprecated. Please use tf.compat.v1.assert_less_equal instead.

W0903 19:38

>> Finished training at 2019-09-03 21:23:00.624834


In [0]:
def input_fn_builder(features, seq_length, is_training, drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  for feature in features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_id)

  def input_fn(params):
    """The actual input function."""
    print(params)
    batch_size = 500

    num_examples = len(features)

    d = tf.data.Dataset.from_tensor_slices({
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "segment_ids":
            tf.constant(
                all_segment_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "label_ids":
            tf.constant(all_label_ids, shape=[num_examples], dtype=tf.int32),
    })

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d

  return input_fn

In [0]:
predict_examples = create_examples(X_test, 'test')

predict_features = run_classifier.convert_examples_to_features(
    predict_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

predict_input_fn = input_fn_builder(
    features=predict_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

result = estimator.predict(input_fn=predict_input_fn)

In [0]:
preds = []
for prediction in result:
      preds.append(np.argmax(prediction['probabilities']))

{}


W0903 22:32:36.219150 139916593944448 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [0]:
from sklearn.metrics import accuracy_score

In [0]:
print("Accuracy of BERT is:",accuracy_score(y_test,preds))

Accuracy of BERT is: 0.8356713426853707


In [0]:
from sklearn.metrics import classification_report
print(classification_report(y_test,preds))

              precision    recall  f1-score   support

           0       0.84      0.84      0.84       258
           1       0.83      0.83      0.83       241

    accuracy                           0.84       499
   macro avg       0.84      0.84      0.84       499
weighted avg       0.84      0.84      0.84       499



In [0]:
pred_sentences = [
 "That movie was absolutely awful",
 "The acting was a bit lacking",
 "The film was creative and surprising",
 "Absolutely fantastic!",
   "Uh....say what?",
   "No words. Haha",
   "Kelli Jackson Let's gooooooo.",
   "NaN",
   "NaN",
   "So proud of this man I know! We used to be cadet children. Now he’s an active commander. Fireworks going off! Joshua Reevesyayyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy",
   "Our son the prodigy",
   "For anyone who wonders what it was like.",
   "Alright friends it’s that time of year again. Aaron Burton and I are doing the Tempe Tri, are you in?",
   "Also my favorite.",
   "Bringing back an oldie but a goodie. Still my favorite drummer. So skilled.",
   "This. Is. Gold.",
   "Maybe someday I can look this cool. Nick Dudzinski",
   "It’s not just us! I am going to kill you",
   "The countdown begins!!!!",
   "Better than morning coffee is this stunna being a drumma.",
   "Matt Wilson your dance move has been stolen!!!!! Kelli Jackson Garrett Joshua Nail Julia Wilson",
   "Wonderful Times!",
   "The food is okay!",
   "hold on that though, it is horrible",
   "come on , i dont want to stay here",
   "Keep it simple, why do I have to tell you always that?",
   "tell me something, how do i carry this on, it seems impossible!",
   "I feel better now, thank you",
   "Please, dont say that!",
   "check that again",
   "how did you adapt so easily, I heart that",
   "The ship has sailed!",
    "I want to puke!!",
    "I want to go use my gun for hunting with my dad, I will have such a nice time",
    "I want to use my gun today at the school",
    "I am feeling depressed",
    "I was cyber bullied today,, noone notice it!!!",
    "I am angry!!!",
    "what the hell!",
    "help donate for cancer",
    "I have cancer",
    "cancer is my sunsign, I was born in september!"
   
]

df3_test = np.asarray(pred_sentences)
df3_test

array(['That movie was absolutely awful', 'The acting was a bit lacking',
       'The film was creative and surprising', 'Absolutely fantastic!',
       'Uh....say what?', 'No words. Haha',
       "Kelli Jackson Let's gooooooo.", 'NaN', 'NaN',
       'So proud of this man I know! We used to be cadet children. Now he’s an active commander. Fireworks going off! Joshua Reevesyayyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy',
       'Our son the prodigy @kellijacks_',
       'For anyone who wonders what it was like.',
       'Alright friends it’s that time of year again. Aaron Burton and I are doing the Tempe Tri, are you in?',
       'Also my favorite.',
       'Bringing back an oldie but a goodie. Still my favorite drummer. So skilled.',
       'This. Is. Gold.',
       'Maybe someday I can look this cool. Nick Dudzinski',
       'Kelli Jackson It’s not just us! I am going to kill you',
       'The countdown begins!!!!',
       'Better than morning coffee is this stunna 

In [0]:
!pip install vaderSentiment
import vaderSentiment

     |████████████████████████████████| 133kB 3.5MB/s 


In [0]:
pred_sentences = [
 "That movie was absolutely awful",
 "The acting was a bit lacking",
 "The film was creative and surprising",
 "Absolutely fantastic!",
   "Uh....say what?",
   "No words. Haha",
   "Kelli Jackson Let's gooooooo.",
   "NaN",
   "NaN",
   "So proud of this man I know! We used to be cadet children. Now he’s an active commander. Fireworks going off! Joshua Reeves",
   "Our son the prodigy @kellijacks_",
   "For anyone who wonders what it was like.",
   "Alright friends it’s that time of year again. Aaron Burton and I are doing the Tempe Tri, are you in?",
   "Also my favorite.",
   "Bringing back an oldie but a goodie. Still my favorite drummer. So skilled.",
   "This. Is. Gold.",
   "Maybe someday I can look this cool. Nick Dudzinski",
   "Kelli Jackson It’s not just us! I am going to kill you",
   "The countdown begins!!!!",
   "Better than morning coffee is this stunna being a drumma.",
   "Matt Wilson your dance move has been stolen!!!!! Kelli Jackson Garrett Joshua Nail Julia Wilson",
   "Wonderful Times!",
   "The food is okay!",
   "hold on that though, it is horrible",
   "come on , i dont want to stay here",
   "Keep it simple, why do I have to tell you always that?",
   "tell me something, how do i carry this on, it seems impossible!",
   "I feel better now, thank you",
   "Please, dont say that!",
   "check that again",
   "how did you adapt so easily, I heart that",
   "The ship has sailed!",
   
]

In [0]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
for sentence in pred_sentences:
    vs = analyzer.polarity_scores(sentence)
    print("{:-<65} {}".format(sentence, str(vs)))

In [0]:
predict_examples_test = create_examples((df3_test), 'test')

predict_features_test = run_classifier.convert_examples_to_features(
    predict_examples_test, label_list, MAX_SEQ_LENGTH, tokenizer)

predict_input_fn = input_fn_builder(
    features=predict_features_test,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

result_test = estimator.predict(input_fn=predict_input_fn)

In [0]:
preds_test = []
for prediction in result_test:
      preds_test.append(np.argmax(prediction['probabilities']))

{}


In [0]:
np.shape(preds_test)
preds_test

[0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0]

In [0]:
emo_sc = {0: 'negative', 1: 'neutral', 2: 'positive'}
sc = [emo_sc[item] for item in preds_test]
sc
for i in range(len(preds_test)):
  print(sc[i],pred_sentences[i])

negative That movie was absolutely awful
negative The acting was a bit lacking
neutral The film was creative and surprising
neutral Absolutely fantastic!
negative Uh....say what?
neutral No words. Haha
neutral Kelli Jackson Let's gooooooo.
negative NaN
negative NaN
neutral So proud of this man I know! We used to be cadet children. Now he’s an active commander. Fireworks going off! Joshua Reevesyayyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
neutral Our son the prodigy @kellijacks_
neutral For anyone who wonders what it was like.
neutral Alright friends it’s that time of year again. Aaron Burton and I are doing the Tempe Tri, are you in?
neutral Also my favorite.
neutral Bringing back an oldie but a goodie. Still my favorite drummer. So skilled.
neutral This. Is. Gold.
neutral Maybe someday I can look this cool. Nick Dudzinski
negative Kelli Jackson It’s not just us! I am going to kill you
neutral The countdown begins!!!!
neutral Better than morning coffee is this stun